## Demo代码

In [1]:
import torch
from torch import nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/Users/lihaibin/anaconda3/envs/rl/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/lihaibin/anaconda3/envs/rl/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <DAC8FDCB-770B-356E-BA9C-E2F40A2AA20E> /Users/lihaibin/anaconda3/envs/rl/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <43889F86-100F-3086-90C3-D4AE08235BA7> /Users/lihaibin/anaconda3/envs/rl/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [7]:
training_data = datasets.FashionMNIST(
    root = "/Users/lihaibin/Workspace/dataset",
    train = True,
    download= True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root = "/Users/lihaibin/Workspace/dataset",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()


## Optimization loop

### 超参数设置

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

优化循环的每次迭代称为一个epoch。

每个时代由两个主要部分组成：

1、训练循环-迭代训练数据集并尝试收敛到最优参数。

2、验证/测试循环-迭代测试数据集以检查模型性能是否有所改善

### Train Loop

In [ ]:
# 损失函数
loss_fn = nn.CrossEntropyLoss()

# 优化器 这里使用了SGD优化器
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        # 计算预测值和标签
        pred = model(x)
        loss = loss_fn(pred, y)

        # 计算梯度反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad() #保证每次梯度都是基于当前的批次进行处理的

        if batch%100==0:
            loss, current = loss.item(), batch * batch_size + len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]\n")


### Test Loop

In [16]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 使用torch.no_grad（）评估模型确保在测试模式期间不计算梯度
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # .type(torch.float) 转化为浮点数，.sum().item()进行求和并获取到对应的python值

    test_loss /= num_batches
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 实际测试结果

In [17]:
# 初始化损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------
loss: 2.146705  [   64/60000]

loss: 2.134158  [ 6464/60000]

loss: 2.075248  [12864/60000]

loss: 2.092306  [19264/60000]

loss: 2.035491  [25664/60000]

loss: 1.982123  [32064/60000]

loss: 1.994717  [38464/60000]

loss: 1.916403  [44864/60000]

loss: 1.920602  [51264/60000]

loss: 1.857001  [57664/60000]

Test Error: 
 Accuracy: 62.4%, Avg loss: 1.849369 

Epoch 2
-----------
loss: 1.883537  [   64/60000]

loss: 1.852181  [ 6464/60000]

loss: 1.730513  [12864/60000]

loss: 1.770056  [19264/60000]

loss: 1.664401  [25664/60000]

loss: 1.626380  [32064/60000]

loss: 1.631855  [38464/60000]

loss: 1.547324  [44864/60000]

loss: 1.574571  [51264/60000]

loss: 1.473835  [57664/60000]

Test Error: 
 Accuracy: 61.6%, Avg loss: 1.491402 

Epoch 3
-----------
loss: 1.562224  [   64/60000]

loss: 1.530811  [ 6464/60000]

loss: 1.380275  [12864/60000]

loss: 1.446049  [19264/60000]

loss: 1.332059  [25664/60000]

loss: 1.338958  [32064/60000]

loss: 1.342159  [38464/60000]


In [4]:
weight_params = [param for name,param in model.named_parameters() if name.endswith('.w')]
bias_params = [param for name,param in model.named_parameters() if name.endswith('.b')]

optimizer =torch.optim.SGD([
                {'params':bias_params},
                {'params': weight_params, 'lr': 1e-2}
            ], lr=1e-5)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

## 部分学习率调整

In [6]:
class perception(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = perception()

如果要调整部分层的学习率，必须从optim中的设置中进行调整

In [11]:
layer1_param = net.fc1.parameters()
base_param = [param for param in net.parameters() if param not in layer1_param]
optim = torch.optim.SGD([
    {'params':layer1_param, 'lr':0.1},
    {'params':base_param}], lr = 0.01)
print(optim)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
